In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd

In [2]:
from tqdm.notebook import *
import tdpkefm
from tdpkefm import *
import glob
import json_tricks

In [3]:
import altair as alt

# for the notebook only (not for JupyterLab) run this command once per session
# alt.renderers.enable('notebook')
alt.data_transformers.enable('default')

DataTransformerRegistry.enable('default')

In [4]:
def get_basic_phase_error(dataset):
    """Grab the data from the json simulation output in a format that will be convenient for making
    a pandas DataFrame."""
    sps = dataset["val"]
    return {'dphi_est_mcyc': sps['dphi_est'], 'percent_error': sps['error']*100, 'dphi_act_mcyc': sps['dphi_est']/(1 + sps['error']),
            'x0': sps['sol'][0, 0]*1000, 'tp': sps['V']['tp'], "Czz_q": sps['sim']['Czz_q'], "DCzz": sps['sim']['DCzz'], "Cs": sps['sim']['Cs']}


# Plot Fig. S3

In [5]:
combined_data = json_tricks.load("results/S13-data.json.gz", conv_str_byte=True)

In [6]:
basic_phase_error_list = {i: process_dataset(x, get_basic_phase_error) for i, x in enumerate(combined_data)}
df = pd.DataFrame.from_dict(basic_phase_error_list, orient='index')

In [7]:
df['err_mcyc'] = df['dphi_est_mcyc'] - df['dphi_act_mcyc']
df_gb = df.groupby(['tau', 'Rlight', 'Rdark', "V0_frac", 'tp', 'DCzz', 'Cs'])
df_sum_1 = df_gb.mean().reset_index()

df_sum_1["𝜏 (µs)"] = df_sum_1["tau"]
df_sum_1["ΔC′′ (pF/nm²)"] = np.round(df_sum_1['DCzz']*1e6, 1)
df_sum_1['C_s (fF)'] = np.round(df_sum_1['DCzz']*1e3, 3)
df['𝜏 (µs)']=df['tau']
df["ΔC′′ (pF/nm²)"] = np.round(df['DCzz']*1e6, 1)

m0 = df['phase'] == 0

In [8]:
expt = alt.Chart(df_sum_1).mark_point().encode(
    x='dphi_est_mcyc',
    y='dphi_act_mcyc',
    color='𝜏 (µs):N',
    shape="ΔC′′ (pF/nm²):N"
)
phase = alt.sequence(0, -12, -0.1, as_='x')
theory = alt.Chart(phase).mark_line(color='gray').encode(x=alt.X('x:Q', title="Predicted Δϕ [mcyc]"),
                                                         y=alt.Y('x:Q', title="Simulated Δϕ [mcyc]"))
together = expt + theory
together

alt.LayerChart(...)

In [9]:
together.save('figs/Fig-S3.png', scale_factor=3.0)

## S3 Residuals

In [10]:
expt = alt.Chart(df_sum_1).mark_point().encode(
    x='tau',
    y='err_mcyc',
    color='DCzz:N',
).interactive()
expt
zero = alt.sequence(0, 100, 10, as_='x')

theory = alt.Chart(zero).mark_line(color='gray').encode(x='x:Q', )
expt + theory


alt.LayerChart(...)

## Summary statistics

In [11]:
# Just the \phi_0 = 0 points:
print(u"For pts with φ₀ = 0")
print("--------------------")
print("Resid. std. dev. = {:.3f}".format(np.std(df[m0]['dphi_act_mcyc'] - df[m0]['dphi_est_mcyc'])))
print("Resid. mean = {:.3f}".format(np.mean(df[m0]['dphi_act_mcyc'] - df[m0]['dphi_est_mcyc'])))

For pts with φ₀ = 0
--------------------
Resid. std. dev. = 0.049
Resid. mean = 0.036


In [12]:
# Just the \phi_0 = 0 points:
print(u"Averaging φ₀ = 0 and π")
print("------------------------")
print("Resid. std. dev. = {:.3f}".format(np.std(df_sum_1['dphi_act_mcyc'] - df_sum_1['dphi_est_mcyc'])))
print("Resid. mean = {:.3f}".format(np.mean(df_sum_1['dphi_act_mcyc'] - df_sum_1['dphi_est_mcyc'])))

Averaging φ₀ = 0 and π
------------------------
Resid. std. dev. = 0.049
Resid. mean = 0.027


In [13]:
print("Largest residual")
print("-----------------")
print("Index: {}".format(np.argmax(df_sum_1['dphi_act_mcyc'] - df_sum_1['dphi_est_mcyc'])))
print(df_sum_1.loc[177])

Largest residual
-----------------
Index: 177
tau              1.000000e+01
Rlight           1.000000e+01
Rdark            1.000000e+08
V0_frac          7.000000e-01
tp               3.840000e+02
DCzz             4.900000e-05
Cs               1.000000e-05
phase            1.570796e+00
dphi_est_mcyc   -9.631036e+00
percent_error    2.074543e+00
dphi_act_mcyc   -9.435297e+00
x0              -7.071068e-01
Czz_q            1.874194e-05
err_mcyc        -1.957386e-01
𝜏 (µs)           1.000000e+01
ΔC′′ (pF/nm²)    4.900000e+01
C_s (fF)         4.900000e-02
Name: 177, dtype: float64


## Other plots

In [14]:
expt = alt.Chart(df).mark_point().encode(
    x='dphi_est_mcyc',
    y='dphi_act_mcyc',
    color='phase:N',
    shape='tp:N'
)
phase = alt.sequence(0, -9, -0.1, as_='x')
theory = alt.Chart(phase).mark_line(color='gray').encode(x='x:Q', y='x:Q')
together = expt + theory
together

alt.LayerChart(...)

In [15]:
expt = alt.Chart(df[m0]).mark_point().encode(
    x='dphi_est_mcyc',
    y='dphi_act_mcyc',
    color='𝜏 (µs):N',
    shape='ΔC′′ (pF/nm²):N'
)
phase = alt.sequence(0, -12, -0.1, as_='x')
theory = alt.Chart(phase).mark_line(color='gray').encode(x='x:Q', y='x:Q')
together = expt + theory
together

alt.LayerChart(...)